In [1]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# #BiocManager::install(c("affyio", "preprocessCore", "zlibbioc", "affy", "limma"), force = TRUE)
# BiocManager::install("vsn")

# #library(vsn)

In [2]:
setwd("/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Data Imputation/Input")
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Data Imputation/Output')
cur_date = "021122"


library(imputeLCMD)
library(tidyverse)
library(janitor)
#library(vsn)

#reading in files
mf_df = read_csv("080321_NoGapFillNoScaling_Dmp3872.csv")
cytokine_df = read_csv("CytokineData_61Subjects_080421.csv")

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:Matrix’:

    which


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
   

Both datasets having missing data that will be imputed using the QRILC method.

# Molecular Feature Dataset Imputation

In [3]:
head(mf_df)

MF_Number,NS_070,NS_110,NS_143,NS_152,NS_187,NS_196,NS_215,NS_216,NS_217,⋯,NA_SB5,NA_SB6,NA_SB7,NA_SC1,NA_SC2,NA_SC3,NA_SC4,NA_SC5,NA_SC6,NA_SC7
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MF_2,1116206475,2608024693,10436125,3733023738,832538895,7117302385,3479543793,886059325,4829905676,⋯,NA,NA,NA,NA,750551.7,487606.7,942984,NA,NA,NA
MF_3,NA,1688508261,5981200,2631091440,NA,5794479646,2855197795,NA,4654806576,⋯,NA,NA,NA,NA,742069.1,NA,389778,NA,418340.2,NA
MF_4,383833198,1374010136,7153567,2629366777,533802873,4533282195,1919783356,353083003,3228249341,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
MF_5,1680099,NA,NA,NA,NA,NA,1573718,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
MF_6,NA,NA,NA,3170821,NA,NA,324521665,NA,7750065,⋯,NA,405819.5,489580.9,NA,289593.1,NA,NA,NA,NA,950554.9
MF_7,129935373,546737402,NA,1191854581,NA,2417054513,1345599792,195011396,2024878820,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [4]:
mf_df = mf_df %>%
    column_to_rownames("MF_Number") %>%
    #removing cols that contain "NA_"
    select(!contains("NA_")) 
head(mf_df)

,NS_070,NS_110,NS_143,NS_152,NS_187,NS_196,NS_215,NS_216,NS_217,NS_220,⋯,Ecig_250,Ecig_251,Ecig_252,Ecig_253,Ecig_254,Ecig_255,Ecig_256,Ecig_257,Ecig_258,Ecig_259
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MF_2,1116206475,2608024693,10436125,3733023738,832538895,7117302385,3479543793,886059325,4829905676,1237297262,⋯,5036756034,5394763618,5973308536,6811671266,6376566779,3882785905,5751112172,4917802454,5333823472,4111940546
MF_3,NA,1688508261,5981200,2631091440,NA,5794479646,2855197795,NA,4654806576,NA,⋯,4896555967,2179633939,3091384429,NA,5842207907,3635883142,5361416545,8596957015,8836674364,2693155019
MF_4,383833198,1374010136,7153567,2629366777,533802873,4533282195,1919783356,353083003,3228249341,625409340,⋯,4463604350,NA,2836145509,3882679868,4604235779,2583451320,5107511143,7923577391,8536154926,2007550662
MF_5,1680099,NA,NA,NA,NA,NA,1573718,NA,NA,NA,⋯,300196925,212750368,10246828,3476769,12098948,1503054162,100067268,5819759405,109366852,505938030
MF_6,NA,NA,NA,3170821,NA,NA,324521665,NA,7750065,NA,⋯,15492861,NA,NA,NA,NA,48679662,22344038,35866638,53865956,NA
MF_7,129935373,546737402,NA,1191854581,NA,2417054513,1345599792,195011396,2024878820,181878763,⋯,2655569279,1355986757,1692335726,2538630126,2488245691,1127673357,2593485817,3644589968,3738572555,1136031318


In [5]:
#Convert mf to binary (if the value is missing 0, if the value is present 1)
mf_missing_NS <- mf_df %>%
    #selecting cols that contain NS
    select(contains("NS")) %>%
    mutate_all(~replace(.,!is.na(.), 1)) %>%
    mutate_all(~replace(., is.na(.), 0)) %>%
    mutate_all(function(x) as.numeric(as.character(x)))
               
 mf_missing_Ecig <- mf_df %>%
    #selecting cols that contain NS
    select(contains("Ecig")) %>%
    mutate_all(~replace(.,!is.na(.), 1)) %>%
    mutate_all(~replace(., is.na(.), 0)) %>%
    mutate_all(function(x) as.numeric(as.character(x)))

In [6]:
#Identify mfs that are detected in at least 50% of NS or Ecig samples
features_to_keep_NS <- mf_missing_NS %>% 
    adorn_totals(where = "col",... = colnames(mf_missing_NS)) %>% 
    filter(Total >= ncol(mf_missing_NS)*.5)
head(features_to_keep_NS) 

features_to_keep_Ecig <- mf_missing_Ecig %>% 
    adorn_totals(where = "col",... = colnames(mf_missing_Ecig)) %>% 
    filter(Total >= ncol(mf_missing_Ecig)*.5)
head(features_to_keep_Ecig) 

,NS_070,NS_110,NS_143,NS_152,NS_187,NS_196,NS_215,NS_216,NS_217,NS_220,⋯,NS_242,NS_243,NS_244,NS_245,NS_246,NS_261,NS_262,NS_263,NS_264,Total
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MF_2,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27
MF_3,0,1,1,1,0,1,1,0,1,0,⋯,1,1,1,1,1,1,1,1,0,20
MF_4,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27
MF_6,0,0,0,1,0,0,1,0,1,0,⋯,1,1,1,1,1,1,1,1,1,17
MF_7,1,1,0,1,0,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,0,24
MF_11,1,1,0,1,0,0,1,0,0,0,⋯,0,1,1,1,1,0,1,1,1,18


,Ecig_39,Ecig_40,Ecig_41,Ecig_44,Ecig_45,Ecig_47,Ecig_48,Ecig_214,Ecig_221,Ecig_222,⋯,Ecig_251,Ecig_252,Ecig_253,Ecig_254,Ecig_255,Ecig_256,Ecig_257,Ecig_258,Ecig_259,Total
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MF_2,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,32
MF_3,1,1,0,1,1,0,0,1,1,1,⋯,1,1,0,1,1,1,1,1,1,27
MF_4,1,1,1,1,1,1,1,1,1,1,⋯,0,1,1,1,1,1,1,1,1,31
MF_5,1,1,1,0,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,29
MF_6,1,0,0,1,0,0,0,0,0,0,⋯,0,0,0,0,1,1,1,1,0,17
MF_7,1,1,0,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,31


In [7]:
#Filter the mf data for the mfs that should be retained based on previous identification step
filtered_mf_matrix = mf_df %>% 
    rownames_to_column("MF") %>%
    filter(MF %in% rownames(features_to_keep_NS) | MF %in% rownames(features_to_keep_Ecig)) %>%
    column_to_rownames("MF") %>%
    t() #%>%
    #can't remember why this is used
    #justvsn()
head(filtered_mf_matrix)

,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,MF_14,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
NS_070,1116206475,NA,383833198,1680099,NA,129935373,194195563,242762512,305851766,8564288.2,⋯,104618.4,85516.80,NA,NA,NA,71508.41,65212.96,NA,NA,NA
NS_110,2608024693,1688508261,1374010136,NA,NA,546737402,753894182,854269800,893795013,4009487.2,⋯,NA,NA,75593.69,NA,30900.99,NA,77846.62,NA,39265.73,40503.16
NS_143,10436125,5981200,7153567,NA,NA,NA,NA,5266245,15765225,1402196.9,⋯,109911.8,80988.63,86084.56,NA,NA,96283.60,NA,NA,47575.70,47844.24
NS_152,3733023738,2631091440,2629366777,NA,3170821,1191854581,1286005199,1650120431,1738876395,756727.8,⋯,121663.8,116516.55,NA,NA,41118.81,93233.71,NA,NA,39667.25,41042.14
NS_187,832538895,NA,533802873,NA,NA,NA,NA,304959865,276521306,12377935.8,⋯,NA,NA,57281.34,NA,20947.90,78186.90,79291.97,NA,41601.24,31705.45
NS_196,7117302385,5794479646,4533282195,NA,NA,2417054513,NA,2723508190,1887555284,3495757.1,⋯,NA,NA,80482.55,NA,NA,NA,NA,NA,NA,NA


In [8]:
#How many molecular features were removed?
nrow(mf_df) - ncol(filtered_mf_matrix)

[1] 0

In [9]:
#QRILC function imputes on normalized, so taking log of data to normalize it
set.seed(7) #ensures reproducibility
imputed_QRILC_object = impute.QRILC(log10(filtered_mf_matrix))
imputed_log_mf_df = data.frame(imputed_QRILC_object[1])
head(imputed_log_mf_df)

,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,MF_14,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,9.047745,8.677792,8.584143,6.225335,6.363655,8.113727,8.288239,8.385182,8.485511,6.932691,⋯,5.019608,4.932051,4.496898,4.680047,3.427012,4.854357,4.814334,4.553534,4.328094,4.168207
NS_110,9.416312,9.227503,9.137990,5.371607,4.473350,8.737779,8.877310,8.931595,8.951238,6.603089,⋯,4.856739,4.804857,4.878486,4.778169,4.489972,4.815671,4.891240,4.540701,4.594014,4.607489
NS_143,7.018539,6.776788,6.854523,5.773716,6.060661,7.908033,8.527189,6.721501,7.197700,6.146809,⋯,5.041044,4.908424,4.934925,4.594672,3.891625,4.983552,4.479880,4.480426,4.677385,4.679830
NS_152,9.572061,9.420136,9.419851,5.730147,6.501172,9.076223,9.109243,9.217516,9.240269,5.878940,⋯,5.085161,5.066388,4.576930,4.560706,4.614041,4.969573,4.531855,4.653725,4.598432,4.613230
NS_187,8.920405,8.891020,8.727381,4.458480,4.544046,8.299618,8.358225,8.484243,8.441729,7.092648,⋯,4.824875,4.839936,4.758013,4.734735,4.321141,4.893134,4.899229,4.639940,4.619106,4.501134
NS_196,9.852315,9.763014,9.656413,4.343602,4.957034,9.383286,8.188907,9.435129,9.275900,6.543541,⋯,4.882482,4.882066,4.905702,4.577540,3.583616,4.456105,4.257600,4.568065,4.420082,4.231063


In [10]:
#coverting data back to the original scale
imputed_mf_df = imputed_log_mf_df %>% 
    mutate_all(., function(x) 10^x)
head(imputed_mf_df)

,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,MF_14,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,1116206475,476202401,383833198,1680098.60,2310229.61,129935373,194195563,242762512,305851766,8564288.2,⋯,104618.45,85516.80,31397.73,47868.13,2673.083,71508.41,65212.96,35771.26,21285.99,14730.15
NS_110,2608024693,1688508261,1374010136,235292.00,29740.65,546737402,753894182,854269800,893795013,4009487.2,⋯,71901.72,63805.39,75593.69,60002.41,30900.989,65414.01,77846.62,34729.67,39265.73,40503.16
NS_143,10436125,5981200,7153567,593903.11,1149902.11,80915782,336658000,5266245,15765225,1402196.9,⋯,109911.82,80988.63,86084.56,39325.32,7791.572,96283.60,30191.20,30229.16,47575.70,47844.24
NS_152,3733023738,2631091440,2629366777,537213.44,3170821.39,1191854581,1286005199,1650120431,1738876395,756727.8,⋯,121663.83,116516.55,37751.11,36366.86,41118.808,93233.71,34029.47,45053.13,39667.25,41042.14
NS_187,832538895,778072467,533802873,28739.57,34998.22,199350676,228152168,304959865,276521306,12377935.8,⋯,66815.16,69172.92,57281.34,54291.91,20947.903,78186.90,79291.97,43645.52,41601.24,31705.45
NS_196,7117302385,5794479646,4533282195,22059.84,90580.45,2417054513,154492300,2723508190,1887555284,3495757.1,⋯,76292.51,76219.52,80482.55,37804.23,3833.679,28582.84,18096.71,36988.39,26307.64,17024.04


# Cytokine Dataset Imputation

In [11]:
head(cytokine_df)

SampleID,Gender,ExposureGroup,ID,IFNg,IL10,IL12p70,IL13,IL1b,IL2,⋯,Eotaxin,Eotaxin3,IL8_1,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CA39,Female,ecig_user,Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,⋯,358.19662,NA,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
CA41,Female,ecig_user,Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
CA44,Female,ecig_user,Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,NA,167.32860,46.98751,101.25635,35.541708
CA45,Female,ecig_user,Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
CA47,Female,ecig_user,Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
CA48,Female,ecig_user,Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [12]:
cytokine_df = cytokine_df %>%
    #removing unecessary metadata cols
    select(!c(SampleID, Gender, ExposureGroup)) %>%
    #removing rows without ids
    filter(is.na(ID)== FALSE) %>%
    column_to_rownames("ID")

head(cytokine_df)

,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,IL8,TNFa,⋯,Eotaxin,Eotaxin3,IL8_1,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,2317.1204,5.9501019,⋯,358.19662,NA,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,2340.2937,11.3628057,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,2233.1268,6.3306852,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,NA,167.32860,46.98751,101.25635,35.541708
Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1889.0348,1.9479068,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,2074.0063,3.5847722,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,823.3077,0.5468069,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [13]:
#There are two IL8 cytokines specifying two ranges of detection. We only want to consider the upper range, 
#thus the lower range is dropped
cyt <- as.data.frame(t(cytokine_df))
cyt <- cyt %>% rownames_to_column("cytokine")
cyt[9,1] <- "IL8_drop"
cyt[23,1] <- "IL8"
cytokine_df <- cyt %>% filter(cytokine!="IL8_drop") %>% column_to_rownames("cytokine")
head(cytokine_df)

,Ecig_39,Ecig_41,Ecig_44,Ecig_45,Ecig_47,Ecig_48,Ecig_214,Ecig_222,Ecig_223,Ecig_229,⋯,NS_217,NS_220,NS_224,NS_227,NS_234,NS_242,NS_245,NS_246,NS_263,NS_264
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IFNg,5.369561,11.612812,4.160319,0.8881171,1.4481966,0.8494440,2.2133006,12.194983,10.359429,11.274641,⋯,1.4100626,2.2885988,1.6232601,12.568316,1.9416090,10.337784,7.890241,0.7175676,7.147723,2.0700327
IL10,2.992854,9.060027,2.034095,0.4310717,0.4997130,0.4240552,1.2942145,4.519250,2.141442,14.467577,⋯,0.3368341,0.6621680,0.4240552,22.625324,0.3994781,4.156594,3.000276,0.1950644,2.339561,0.3868263
IL12p70,1.670016,3.371081,1.565569,0.3063518,0.8622316,0.2785117,0.7038064,5.050170,1.880607,4.079984,⋯,0.5284839,0.7517281,0.8180437,3.124857,0.5377213,4.800905,1.950156,0.2952182,1.814701,0.5303315
IL13,48.368132,67.963220,50.771205,25.2390558,36.8147135,14.5127220,34.6004791,83.282335,61.603704,85.073378,⋯,10.9902839,31.6987943,33.3564823,94.411251,13.3227644,89.613874,69.582659,20.0421143,59.211105,23.3918189
IL1b,24.117761,167.431877,8.870868,40.4776501,15.7558575,9.2116799,53.3450039,98.602727,92.383512,180.302138,⋯,9.4157549,21.4534240,27.8493328,927.559496,11.5052411,89.453532,104.084531,22.8313411,53.797197,17.8222380
IL2,5.265719,12.683771,2.483751,0.5200777,0.7766565,0.2684367,1.8460707,9.060809,4.800959,11.239145,⋯,0.7175273,0.8714269,0.2814659,14.667649,0.7407722,8.725433,5.653496,0.4216734,4.668523,0.3614672


In [14]:
#Convert cytokine to binary (if the value is missing 0, if the value is present 1)
cytokine_missing_NS <- cytokine_df %>%
    #selecting cols that contain NS
    select(contains("NS")) %>%
    mutate_all(~replace(.,!is.na(.), 1)) %>%
    mutate_all(~replace(., is.na(.), 0)) %>%
    mutate_all(function(x) as.numeric(as.character(x)))
               
 cytokine_missing_Ecig <- cytokine_df %>%
    #selecting cols that contain NS
    select(contains("Ecig")) %>%
    mutate_all(~replace(.,!is.na(.), 1)) %>%
    mutate_all(~replace(., is.na(.), 0)) %>%
    mutate_all(function(x) as.numeric(as.character(x)))

In [15]:
#Identify cytokines that are detected in at least 50% of NS or Ecig samples
features_to_keep_NS <- cytokine_missing_NS %>% 
    adorn_totals(where = "col",... = colnames(cytokine_missing_NS)) %>% 
    filter(Total >= ncol(cytokine_missing_NS)*.5)
head(features_to_keep_NS) 

features_to_keep_Ecig <- cytokine_missing_Ecig %>% 
    adorn_totals(where = "col",... = colnames(cytokine_missing_Ecig)) %>% 
    filter(Total >= ncol(cytokine_missing_Ecig)*.5)
head(features_to_keep_Ecig) 

,NS_070,NS_110,NS_143,NS_152,NS_196,NS_225,NS_228,NS_231,NS_233,NS_238,⋯,NS_220,NS_224,NS_227,NS_234,NS_242,NS_245,NS_246,NS_263,NS_264,Total
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IFNg,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27
IL10,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27
IL12p70,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27
IL13,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27
IL1b,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27
IL2,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,27


,Ecig_39,Ecig_41,Ecig_44,Ecig_45,Ecig_47,Ecig_48,Ecig_214,Ecig_222,Ecig_223,Ecig_229,⋯,Ecig_237,Ecig_239,Ecig_240,Ecig_241,Ecig_247,Ecig_250,Ecig_254,Ecig_257,Ecig_259,Total
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IFNg,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,32
IL10,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,32
IL12p70,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,32
IL13,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,32
IL1b,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,32
IL2,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,32


In [16]:
#Filter the cytokine data for the cytokines that should be retained based on previous identification step
filtered_cytokine_matrix = cytokine_df %>% 
    rownames_to_column("cytokine") %>%
    filter(cytokine %in% rownames(features_to_keep_NS) | cytokine %in% rownames(features_to_keep_Ecig)) %>%
    column_to_rownames("cytokine") %>%
    t()
head(filtered_cytokine_matrix)

,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,TNFa,GMCSF,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,0.1302776,⋯,358.19662,NA,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,11.3628057,1.5269950,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,0.7731633,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,NA,167.32860,46.98751,101.25635,35.541708
Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,0.2527908,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,0.2845560,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,0.0571680,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [19]:
#How many cytokines were removed?
nrow(cytokine_df) - ncol(filtered_cytokine_matrix)

[1] 0

In [17]:
#QRILC function imputes on normalized, so taking log of data to normalize it
set.seed(7) #ensures reproducibility
imputed_QRILC_object = impute.QRILC(log10(filtered_cytokine_matrix))
imputed_log_cytokine_df = data.frame(imputed_QRILC_object[1])
head(imputed_log_cytokine_df)

,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,TNFa,GMCSF,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_39,0.72993879,0.4760855,0.22272067,1.684559,1.3823370,0.7214577,-0.3720623,1.1164640,0.7745244,-0.8851303,⋯,2.554121,-1.2087502,4.268291,3.941348,2.051085,1.00311922,1.500963,1.3429488,1.670252,1.3126280
Ecig_41,1.06493738,0.9571295,0.52776919,1.832274,2.2238381,1.1032484,-0.1467287,1.2794930,1.0554856,0.1838376,⋯,2.426763,1.8404625,5.038406,3.886943,2.407040,1.11426383,2.345514,1.7215810,2.173251,1.3929639
Ecig_44,0.61912662,0.3083712,0.19467212,1.705617,0.9479661,0.3951081,-0.3040811,0.9669098,0.8014507,-0.1117288,⋯,2.014964,0.9407422,4.050037,3.851832,2.169478,0.03758596,2.223570,1.6719824,2.005422,1.5507383
Ecig_45,-0.05152978,-0.3654505,-0.51377958,1.402073,1.6072153,-0.2839318,-1.5354317,0.8059006,0.2895682,-0.5972387,⋯,2.475454,0.9262891,4.165357,3.483729,1.981485,1.32634202,2.292531,1.5170673,1.396602,1.2946530
Ecig_47,0.16082753,-0.3012793,-0.06437605,1.566021,1.1974420,-0.1097710,-1.0524292,0.7197042,0.5544616,-0.5458322,⋯,1.827269,0.7393492,4.082001,2.999103,1.820423,0.85672608,2.144455,1.2229406,1.329531,0.9949933
Ecig_48,-0.07086526,-0.3725776,-0.55515650,1.161749,0.9643388,-0.5711580,-0.8502757,0.3939092,-0.2621660,-1.2428470,⋯,1.399238,0.3327558,3.688481,2.392694,1.759694,0.52631120,1.841090,0.9806545,1.195102,0.9686023


In [18]:
#coverting data back to the original scale
imputed_cytokine_df = imputed_log_cytokine_df %>% 
    mutate_all(., function(x) 10^x)
head(imputed_cytokine_df)

,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,TNFa,GMCSF,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,0.1302776,⋯,358.19662,0.06183719,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,11.3628057,1.5269950,⋯,267.15489,69.25681747,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,0.7731633,⋯,103.50569,8.72453305,11221.13,7109.3784,147.73303,1.090400,167.32860,46.98751,101.25635,35.541708
Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,0.2527908,⋯,298.85050,8.43896238,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,0.2845560,⋯,67.18448,5.48717948,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,0.0571680,⋯,25.07486,2.15157171,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [16]:
#exporting data
write.csv(imputed_cytokine_df, paste0(Output,"/", cur_date, "_imputed_cytokine_data_w_Ecig40.csv"), row.names = TRUE)
write.csv(imputed_mf_df, paste0(Output,"/", cur_date, "_imputed_molecular_feature_data_w_Ecig40.csv"), row.names = TRUE)